In [9]:
from astropy.table import Table
from scipy.stats import zscore
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
import sys
import os
from scipy.stats import gaussian_kde
import numpy as np

In [3]:
df_h = Table.read('./table-4LAC-DR3-h.fits') 
df_h = df_h.to_pandas()
df_h = df_h[df_h['Flags']==0]

df_l = Table.read('./table-4LAC-DR3-l.fits') 
df_l = df_l.to_pandas()
df_l = df_l[df_l['Flags']==0]

In [7]:
df_h.head()

,Source_Name,DataRelease,RAJ2000,DEJ2000,GLON,GLAT,Signif_Avg,Flux1000,Unc_Flux1000,Energy_Flux100,...,HE_EPeak,Unc_HE_EPeak,HE_nuFnuPeak,Unc_HE_nuFnuPeak,nu_syn,nuFnu_syn,Variability_Index,Frac_Variability,Unc_Frac_Variability,Highest_energy
0,b'4FGL J0001.2+4741',1,0.3126,47.685902,114.250198,-14.338059,4.692600,1.259796e-10,2.877197e-11,1.499454e-12,...,NaN,NaN,-inf,-inf,1.000000e+14,3.716456e-13,25.313953,0.675882,0.312850,65.575203
1,b'4FGL J0001.2-0747',1,0.3151,-7.797100,89.032722,-67.305008,23.782486,7.471219e-10,5.041939e-11,8.339171e-12,...,796.297852,766.797424,8.686245e-07,7.492923e-08,9.120108e+13,1.936770e-12,46.780693,0.406565,0.111870,86.975197
2,b'4FGL J0001.4-0010',3,0.3717,-0.169900,96.891991,-60.491329,4.900018,1.082246e-10,2.914440e-11,1.231385e-12,...,15475.512695,18471.714844,1.494930e-07,8.294155e-08,3.609939e+12,2.473256e-13,9.272764,0.000000,10.000000,30.056200
3,b'4FGL J0001.5+2113',1,0.3815,21.218300,107.649353,-40.167652,54.653084,1.347354e-09,6.172809e-11,2.555889e-11,...,65.680008,18.915735,8.844835e-06,8.229393e-07,1.584893e+14,1.152471e-12,1910.935791,0.996138,0.213478,-inf
5,b'4FGL J0001.8-2153',3,0.4647,-21.886499,55.184696,-77.572540,4.578068,9.047530e-11,2.442527e-11,1.061114e-12,...,6281.065430,2626.445068,1.545364e-07,5.278574e-08,1.659588e+13,9.716011e-13,24.557972,0.902851,0.380107,-inf


In [6]:
df_l

,Source_Name,DataRelease,RAJ2000,DEJ2000,GLON,GLAT,Signif_Avg,Flux1000,Unc_Flux1000,Energy_Flux100,...,HE_EPeak,Unc_HE_EPeak,HE_nuFnuPeak,Unc_HE_nuFnuPeak,nu_syn,nuFnu_syn,Variability_Index,Frac_Variability,Unc_Frac_Variability,Highest_energy
0,b'4FGL J0012.0+7043',1,3.003200,70.732597,119.661102,8.111637,9.391647,4.022406e-10,4.989643e-11,5.202780e-12,...,NaN,NaN,-inf,-inf,5.754399e+12,5.482871e-13,18.997129,0.381762,0.192456,-inf
1,b'4FGL J0014.7+5801',1,3.686700,58.027199,118.074745,-4.498106,10.829500,1.763550e-10,4.287831e-11,2.571515e-12,...,30467.089844,17695.964844,5.071456e-07,1.101557e-07,0.000000e+00,0.000000e+00,11.195417,0.099530,0.650253,160.817001
2,b'4FGL J0014.8+6118',1,3.721200,61.303699,118.557297,-1.257189,17.677473,7.848316e-10,7.590142e-11,6.474739e-12,...,5067.017578,1501.950317,9.392241e-07,9.292127e-08,2.089296e+13,1.480430e-12,12.274307,0.000000,10.000000,-inf
3,b'4FGL J0015.6+5551',1,3.907100,55.863602,117.888100,-6.657064,13.679805,3.372334e-10,4.716959e-11,4.016745e-12,...,NaN,NaN,-inf,-inf,5.688530e+15,7.878908e-12,11.952563,0.136779,0.318575,167.578003
6,b'4FGL J0035.9+5950',1,8.982300,59.833401,120.982048,-2.979620,68.426636,3.081422e-09,9.592344e-11,3.706086e-11,...,98809.250000,65159.152344,5.909368e-06,5.170865e-07,1.122018e+17,1.493479e-11,252.022751,0.494519,0.109907,372.923004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,b'4FGL J2237.4+5124',3,339.373199,51.415001,102.684616,-6.113458,5.600540,1.486060e-10,4.146037e-11,1.650939e-12,...,4748.909180,5513.632812,1.636098e-07,4.349667e-08,3.311315e+13,1.006315e-12,14.602383,0.328630,0.612450,97.652802
398,b'4FGL J2251.2+5550',2,342.803894,55.843102,106.574341,-3.163003,5.120801,2.168504e-10,6.168643e-11,1.282844e-12,...,3268.443604,1497.775391,2.823202e-07,6.913375e-08,0.000000e+00,0.000000e+00,34.235767,1.071415,0.315385,-inf
402,b'4FGL J2329.7+6101',1,352.434296,61.023899,113.198418,-0.294415,15.066041,6.746162e-10,7.522848e-11,7.496495e-12,...,3268.194580,9568.627930,6.829078e-07,7.403928e-08,0.000000e+00,0.000000e+00,22.359163,0.337927,0.140322,143.735001
403,b'4FGL J2347.0+5141',1,356.765900,51.696602,112.886703,-9.915193,70.645012,2.882701e-09,8.283773e-11,3.227763e-11,...,56117.906250,39034.074219,4.268890e-06,3.307706e-07,1.566751e+16,1.279994e-11,58.688427,0.201833,0.052628,805.708008


In [10]:
classification_df = pd.concat([df_h, df_l], ignore_index=True)
classification_df['CLASS'] = classification_df['CLASS'].str.decode('utf-8')
classification_df['CLASS'] = classification_df['CLASS'].str.upper()
classification_df = classification_df[classification_df['CLASS'].isin(['BLL', 'FSRQ', 'BCU'])]

selected_columns = ['PL_Index', 'nu_syn', 'LP_Index', 'Pivot_Energy', 'Frac_Variability', 'Variability_Index', 'nuFnu_syn', 'CLASS']
target = 'CLASS'
feature_columns = [ele for ele in selected_columns if ele is not 'CLASS']


classification_df = classification_df[selected_columns]

classification_df

<>:8: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:8: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_8057/3147434928.py:8: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  feature_columns = [ele for ele in selected_columns if ele is not 'CLASS']


,PL_Index,nu_syn,LP_Index,Pivot_Energy,Frac_Variability,Variability_Index,nuFnu_syn,CLASS
0,2.271696,1.000000e+14,2.254081,2420.575195,0.675882,25.313953,3.716456e-13,BCU
1,2.116692,9.120108e+13,2.078927,1721.603882,0.406565,46.780693,1.936770e-12,BLL
2,1.939160,3.609939e+12,1.661223,4307.082520,0.000000,9.272764,2.473256e-13,BLL
3,2.654060,1.584893e+14,2.514159,329.773743,0.996138,1910.935791,1.152471e-12,FSRQ
4,1.876663,1.659588e+13,1.716620,4429.934082,0.902851,24.557972,9.716011e-13,BCU
...,...,...,...,...,...,...,...,...
3169,2.071567,3.311315e+13,2.042366,5897.854004,0.328630,14.602383,1.006315e-12,BCU
3170,2.226648,0.000000e+00,1.962429,3114.226318,1.071415,34.235767,0.000000e+00,BCU
3171,2.045913,0.000000e+00,2.019132,5066.525391,0.337927,22.359163,0.000000e+00,BCU
3172,1.810977,1.566751e+16,1.736923,1910.885742,0.201833,58.688427,1.279994e-11,BLL
